In [ ]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=c9d2999f800a925ad1d0c5c26d5d5b829b865d9f09803f36c13c1df168d1563a
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [ ]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [ ]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [ ]:
# calling API 
data=YahooFinancials('BTC-USD').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
btc_df = pd.DataFrame(data['BTC-USD']['prices'])

btc_df =btc_df .drop(['high','low','open','volume','date','adjclose'], axis=1)
btc_df.rename(columns={'formatted_date':'date'},inplace=True)
btc_df['date']=pd.to_datetime(btc_df['date'])

In [ ]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [ ]:
btc_df

,close,date
0,9267.561523,2019-11-07
1,8804.880859,2019-11-08
2,8813.582031,2019-11-09
3,9055.526367,2019-11-10
4,8757.788086,2019-11-11
...,...,...
1096,20602.816406,2022-11-07
1097,18541.271484,2022-11-08
1098,15880.780273,2022-11-09
1099,17586.771484,2022-11-10


In [ ]:
btc_df.dtypes

close           float64
date     datetime64[ns]
dtype: object

In [ ]:
# Scale data for ML engine
scaler = MinMaxScaler()
btc_df['scaled_close'] = scaler.fit_transform(np.expand_dims(btc_df['close'].values, axis=1))
# df = btc.copy()
# df['future'] = df['scaled_close'].shift(-3)
# last_sequence = np.array(df[['scaled_close']].tail(3))
# last_sequence

In [ ]:
def PrepareData(days):
    df = btc_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [ ]:
PrepareData(3) # 3 days


(             close       date  scaled_close    future
 0      9267.561523 2019-11-07      0.068643  0.065256
 1      8804.880859 2019-11-08      0.061251  0.060499
 2      8813.582031 2019-11-09      0.061390  0.061424
 3      9055.526367 2019-11-10      0.065256  0.061305
 4      8757.788086 2019-11-11      0.060499  0.059705
 ...            ...        ...           ...       ...
 1093  21147.230469 2022-11-04      0.258426  0.249729
 1094  21282.691406 2022-11-05      0.260590  0.216795
 1095  20926.486328 2022-11-06      0.254899  0.174292
 1096  20602.816406 2022-11-07      0.249729  0.201546
 1097  18541.271484 2022-11-08      0.216795  0.194450
 
 [1098 rows x 4 columns], array([[0.24264659],
        [0.2434531 ],
        [0.25842598],
        [0.26059002],
        [0.25489947],
        [0.24972871],
        [0.2167946 ],
        [0.17429206],
        [0.20154603],
        [0.19445027]], dtype=float32), array([[[0.06864289553809853, Timestamp('2019-11-07 00:00:00')],
         [0

In [ ]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [ ]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
137/137 [==============================] - 6s 13ms/step - loss: 0.0093
Epoch 2/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0032
Epoch 3/80
137/137 [==============================] - 3s 24ms/step - loss: 0.0032
Epoch 4/80
137/137 [==============================] - 3s 24ms/step - loss: 0.0029
Epoch 5/80
137/137 [==============================] - 3s 23ms/step - loss: 0.0027
Epoch 6/80
137/137 [==============================] - 3s 23ms/step - loss: 0.0026
Epoch 7/80
137/137 [==============================] - 3s 24ms/step - loss: 0.0022
Epoch 8/80
137/137 [==============================] - 3s 23ms/step - loss: 0.0021
Epoch 9/80
137/137 [==============================] - 3s 23ms/step - loss: 0.0019
Epoch 10/80
137/137 [==============================] - 2s 18ms/step - loss: 0.0017
Epoch 11/80
137/137 [==============================] - 2s 13ms/step - loss: 0.0021
Epoch 12/80
137/137 [==============================] - 2s 12ms/step - loss: 0.0020
Epoch 13/80
1

In [ ]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (17181.64$, 18164.22$, 16897.49$)


In [15]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
btc_lstm=pd.DataFrame()
btc_lstm['future days']=days
btc_lstm['price']=predictions
btc_lstm['ticker']='BTC'

btc_lstm.to_csv('btc_lstm.csv') 
files.download('btc_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>